# 3.2 - Pandas Deepdive

![pandas_deep](../../images/pandas_deep.jpg)

Antes de empezar con las técnicas de limpieza de datos y visualización vamos a ver como cargar los diferentes tipos de datos con los que podemos trabajar

#### Tipos de archivos con los que podemos trabajar en pandas

    - CSV : 'Comma Separated Values' son archivos de texto plano con los valores separados por comas, si está correctamente codificado pandas interpretará como una tabla y cada una de las comas corresponderá a un dato de una columna, también pueden abrirse desde excel y este los intepretará como una tabla de la misma forma
    - xlsx & xls: 'Archivos de excel' importa los datos de un archivo de excel con la salvedad de que únicamente importa los datos de las filas y columnas, pero no las fórmulas que hayamos utilizado para obtenerlos.
    - json: JSON son las siglas de "JavaScript Object Notation". Un archivo JSON, tiene como extensión los .json, además los datos que contiene son representados en un par llave:valor, igualmente que un objeto JavaScript tradicional. Por simplificar un archivo json podemos interpretarlo como un diccionario de python.



**Para poder usar archivos de excel en python deberemos de instalar las siguientes librerías**

In [ ]:
%pip install openpyxl
%pip install xlrd

**Importamos las librerías necesarias**

In [ ]:
import pandas as pd
import numpy as np

**Cargar un archivo .csv**

In [ ]:
df_csv = pd.read_csv('../../../data/Marketing-Customer-Analysis.csv')

df_csv.head()

**Cargar un archivo .xlsx o .xls**

In [ ]:
df_excel = pd.read_excel('../../../data/retail.xlsx')

df_excel.head()

**Cargar un archivo .json**

In [ ]:
df_json = pd.read_json('../../../data/companies.json', orient='records', lines=True)

df_json.head()

**También podemos cargar datos desde un archivo zip**

In [ ]:
import zipfile

In [ ]:
zp = zipfile.ZipFile('../../../data/mahindra.zip')

df_zip = pd.read_csv(zp.open('test.csv'), nrows=1e5)

df_zip.head()

**Guardar datos**

    Al igual que podemos importar datos podemos exportar nuestros dataframes a diferentes tipos de datos

In [ ]:
df_csv.to_excel('../../../data/Marketing-Customer-Analysis.xlsx', index=False)
df_excel.to_json('../../../data/retail.json', orient='table')
df_json.to_csv('../../../data/companies.csv', index=False) 
#el parametro index=False lo usamos para evitar que nos guarde el índice de df como una columna adicional

## Data Cleaning


![cleaning](../../images/cleaning.jpg)


$$$$

Una vez que hemos conseguido los datos necesarios, es necesario observar su calidad y orden. No se trata de un análisis exploratorio de los datos, sino de ver como de sucios y desordenados están nuestros datos. Cuando los datos tienen problemas de calidad decimos que los datos están sucios. Los problemas de calidad están relacionados con valores nulos, datos inconsistentes, tipo de dato incorrecto y registros duplicados. 


$$$$
![assess](../../images/data_assess.png)
$$$$
$$$$

Por otro lado, el orden de los datos está relacionado con su estructura. Como dice [Hadley Wickham](http://hadley.nz/) en su paper [Tidy Data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html), los datos están ordenados cuando:

+ Cada variable forma una columna.
+ Cada observación forma una fila.
+ Cada tipo de unidad obsservacional forma una tabla.
$$$$
![clean](../../images/data_clean.png)

**Importamos las librerias necesarias**

In [ ]:
%pip install ipython
%pip install seaborn

In [ ]:
# para quitar warnings
import warnings
warnings.filterwarnings('ignore')  

# muestra todas las columnas del df.head()
pd.set_option('display.max_columns', None)

# muestra todas las filas del df
#pd.set_option('display.max_rows', None)    

# para hacer gráficos
import pylab as plt
import seaborn as sns

# para que salga el grafico
%matplotlib inline

In [ ]:
df = pd.read_csv('../../../data/vehicles_messy.csv')

df.head()

### Exploración

Generamos una variable con las dimensiones iniciales de nuestro df para poder comparar con el resultado final

In [ ]:
dim_ori = df.shape 
dim_ori

**Para ver las columnas de nuestro df**

In [ ]:
df.columns

**Vamos a ir explorando algunas de las columnas para ver como están nuestros datos**

In [ ]:
df.info()

**Podemos ver cuantas columnas tenemos con datos númericos y cuantas categóricas**

In [ ]:
# numéricas

In [ ]:
num_df = df.select_dtypes(exclude='object')

num_df.columns

In [ ]:
len(num_df.columns)

In [ ]:
# categóricas

In [ ]:
cat_df = df.select_dtypes(include='object')

cat_df.columns

In [ ]:
len(cat_df.columns)

**Antes de comenzar nuestro proceso de análisis y limpieza podemos ver una descripción general de nuestro df**

In [ ]:
df.describe().T

**Comenzamos nuestro análisis explorando las columnas con datos nulos**

Usaremos el método isna() de pandas para comprobar el número de datos nulos que hay en cada columnas y realizamos una suma de los mismos

In [ ]:
nan_cols = df.isna().sum()

Filtramos nuestro df para ver cuales son las columnas que tienen datos nulos

In [ ]:
nan_cols[nan_cols>0]

**También podemos hacer el cálculo y mostrarlo como una tasa**

In [ ]:
nan_cols = df.isna().mean()*100

nan_cols[nan_cols>0]

**O mediante un gráfico**

In [ ]:
# grafica de nulos en el dataframe

plt.figure(figsize=(20,12))

sns.heatmap(df.isna(),
           yticklabels=False,
           cmap='viridis',
           cbar = False)

plt.show(); # poner ';' para evitar que imprima la dirección de memoria del objeto que estamos pintando

Vamos a encapsular en una función el código anterior por si necesitamos el gráfico más adelante y poder reutilzarlo.

In [ ]:
def plt_isna(df:pd.DataFrame)->None:
    
    '''
        Función que recibe un dataframe como parámetro y devuelve un heatmap con los datos nulos
        de cada una de sus columnas
        
        Parametros.
        -----------
        df = Pandas dataframe
        
        Return.
        -----------
        
        La función muestra por pantalla un heatmap con los datos nulos por cada columna, pero no devuelve nada
    
    '''
    
    plt.figure(figsize=(20,12))

    sns.heatmap(df.isna(),
           yticklabels=False,
           cmap='viridis',
           cbar = False)

    plt.show(); # poner ';' para evitar que imprima la dirección de memoria del objeto que estamos pintando
    

In [ ]:
plt_isna(df)

Si una columna tiene muchos valores nulos puede ensuciar nuestro análisis e inducirnos a error por lo que si tiene más del 50-60% de sus valores nulos, un opción bastante viable es eliminarla

**Vamos a borrar las columnas con más de 10000 valores nulos**

In [ ]:
nan_cols = df.isna().sum()
nan_cols[nan_cols>1e4].index

**Antes de comenzar la limpieza es bueno realizar una copia de seguridad de nuestros datos originales**

In [ ]:
df_original = df.copy()

Una vez hecha la copia de seguridad comenzamos el proceso de limpieza

In [ ]:
df.drop(columns=nan_cols[nan_cols>1e4].index, inplace=True)

In [ ]:
df.shape, df_original.shape

In [ ]:
plt_isna(df)

In [ ]:
nan_cols = df.isna().sum()
nan_cols[nan_cols>0]

Una vez eliminadas las columnas con mayor número de nulos vamos a proceder a limpiar las columnas que aún tienen datos nulos

### Columna Cylinders & displ

Lo primero que vamos a hacer será localizar los índices de la columna que tienen valores nulos para ver si podemos sacar alguna conclusión de porque nos falta ese dato y si podemos rellenarlo de alguna manera

In [ ]:
bad_index = df.cylinders[df.cylinders.isna()].index

bad_index[:10]

Una vez que tenemos los índices vamos a filtrar nuestro df por esos índices y seleccionaremos una serie de columnas a ver si nos puede ayudar a entender porque nos falta ese dato.

En nuestro caso vamos a seleccionar las columnas de 'make', 'model', 'fuelType', 'cylinders', 'displ'

In [ ]:
df.iloc[bad_index][['make', 'model', 'fuelType', 'cylinders', 'displ']].head()

En este caso particular parece que la columna fuelType puede ayudarnos a entender el porque la columna cylinders tiene un dato nulo

In [ ]:
df.iloc[bad_index][['make', 'model', 'fuelType', 'cylinders', 'displ']].fuelType.unique()

Como podemos observar todos los coches que tenemos aquí tienen motor eléctrico o 'regular', los coches con motor eléctrico no tienen cilindros por ese motivo nos falta ese dato, vamos a ver si podemos entender que marcas tienen el motor 'regular'

In [ ]:
df.iloc[bad_index][['make', 'model', 'fuelType', 'cylinders', 'displ']][df.fuelType=='Regular']

**¿Qué es un motor rotativo?
[wikipedia](https://es.wikipedia.org/wiki/Motor_rotativo)

Como vemos tanto los motores eléctricos como los rotativos no tienen cilindros, por lo que podemos rellenar nuestros campos con un 0 y de esta manera ya tendríamos esta columna limpia.

Para ello utilizaremos el método de pandas fillna()

In [ ]:
df[['cylinders', 'displ']] = df[['cylinders', 'displ']].fillna(0) 

In [ ]:
df.iloc[bad_index][['make', 'model', 'fuelType', 'cylinders', 'displ']].head()

### Columna drive

In [ ]:
bad_index = df.drive[df.drive.isna()].index

bad_index[:10]

Vamos a seguir el mismo procedimiento que con las columnas cilynders y displ

In [ ]:
df.iloc[bad_index][['make', 'model', 'fuelType', 'drive']].head()

**Este es el punto donde siempre hay que decidir. ¿Qué nos conviene más, borrar registros o rellenar con un `unknown`?**

Para ayudarnos a decidir vamos a ver que tipos drive tenemos en nuestro dataframe

In [ ]:
df.drive.unique()

En este caso y como el volumen de datos incompletos es alto en vez de borrarlos vamos a proceder a rellenar los datos con 'unknown' y de esta manera evitamos perder casi 1200 registros.

In [ ]:
df.drive.fillna('unknown', inplace=True)

In [ ]:
nan_cols = df.isna().sum()

nan_cols[nan_cols > 0]

### Columna trany

In [ ]:
bad_index = df.trany[df.trany.isna()].index

df.iloc[bad_index][['make', 'model', 'fuelType', 'trany']].head()

In [ ]:
df.trany.fillna('unknown', inplace=True)

In [ ]:
nan_cols = df.isna().sum()

In [ ]:
plt_isna(df)

**Ahora vamos a proceder a la eliminación de datos duplicados**

Para ello tenemos el método de pandas drop_duplicates()

Antes de proceder a eliminarlos comprobaremos si realmente hay duplicados o no.

In [ ]:
df.drop_duplicates().shape, df.shape

En principio no parece que tengamos datos duplicados

### Columnas constantes o con baja varianza

Primero nos fijaremos en las columnas numéricas, porque hacemos un estudio de esto, una columna constante o con una varianza baja no nos aporta valor a nuestro análisis y puede introducirnos ruido en nuestro posterior modelo, por lo que este tipo de columnas suelen eliminarse de los análisis.

In [ ]:
# columnas ctes

cte_cols = []

for c in df.select_dtypes(include=np.number): # para columnas de tipo numérico
    
    if df[c].std()==0: #si la desviación standar de una columna es 0 significa que su valor es cte
        cte_cols.append(c)
        
cte_cols

In [ ]:
df.charge120.value_counts()

In [ ]:
df.drop(cte_cols, axis=1, inplace=True)

In [ ]:
cte_cols = []

for c in df.select_dtypes(include=np.number): # para columnas de tipo numérico
    
    if df[c].std()==0:
        cte_cols.append(c)
        
cte_cols

In [ ]:
cte_str_cols = []

for c in df.select_dtypes(include='object'): # para columnas de tipo No numérico
    
    if len(df[c].unique())==1: #si la columna categorica solo tiene un valor no indica que esta es cte
        cte_str_cols.append(c)
        
cte_str_cols

In [ ]:
df.shape, df_original.shape

### Outliers (datos atípicos)

Outlier se refiere a esos registros con valores extremos, fuera del rango intercuartil. Dichos valores podrían ser datos verdaderos o fallos de registro. Sea como fuere, se suelen eliminar estos registros porque podrían desplazar la media de una manera ficticia. Esto es, la robustez de los datos. Los datos atípicos 'pesan más' que los datos cercanos a la media. Un solo valor es suficiente para influenciar enormemente la media del conjunto de datos. Esta vez nos fijaremos en los datos por filas.

In [ ]:
#stats

stats = df.describe().T

stats['IQR'] = stats['75%']-stats['25%']

stats.tail()

In [ ]:
df[['youSaveSpend']].boxplot(figsize=(10,6));

Vamos a generar una función que nos calcule el índice IQR de cada una de nuestras

In [ ]:
def get_outliers(stats:pd.DataFrame, threshold: float = 1.5)-> pd.DataFrame:
    '''
        Esta función recibe un dataframe del tipo df.describe().T
        
        Nos devuelver un dataframe con los outliers
        
        Params:
            stats = pd.DataFrame.describe().T
            threshold = float, con el umbral de tukey
        
        Return:
            pd.DataFrame
    
    '''
    
    stats['IQR'] = stats['75%']-stats['25%']
    
    outliers = pd.DataFrame(columns=stats.index)
    
    for c in stats.index:
        iqr = stats.at[c, 'IQR']
        
        cut_off = threshold * iqr 
        
        lower = stats.at[c, '25%'] - cut_off
        upper = stats.at[c, '75%'] + cut_off
        
        res = df[(df[c] < lower) | (df[c] > upper)].copy()
        
        res['outliers'] = c
        
        outliers = outliers.append(res, sort=True)
    
    print(outliers['outliers'].value_counts())
    return outliers

In [ ]:
outliers = get_outliers(df.describe().T)

In [ ]:
outliers.head()

Ahora vamos extraer los índices de este df en una lista para filtrar nuestro df y de esta manera quitar los datos atípicos de nuestro df.

**¡¡¡OJO!!!!** Solo se quitan outliers una sola vez, ya que siempre habrá datos que superen los límites de "normalidad" y corremos el riesgo de quedarnos sin datos.

In [ ]:
clean_index = [i for i in df.index if i not in outliers.index]

In [ ]:
df = df.iloc[clean_index]

In [ ]:
df[['youSaveSpend']].boxplot(figsize=(10,8))

### Colinealidad

    A grosso modo podemos decir que existe colinealidad entre dos columnas cuando ambas nos están aportando la misma información, bien porque sean la misma o porque una es combinación lineal de la otra
    
    Para estudiar la colinealidad lo haremos a través de la matriz de correlación, si en esta encontramos valores nulos signifacará que la varianza entre ambas columnas es 0.
    



$$\rho(X,Y)=\frac{\sigma{X,Y}}{\sigma{X}\sigma{Y}} =\frac{Cov(X,Y)}{\sqrt{ Var{(X)} Var{(Y)}}}$$



$$\rho{(X,Y)}$$ es la correlación de pearson

$$\sigma{(X,Y)}$$ es la covarianza de (X,Y)

$$\sigma{(X)}$$ es la desviación estándar de (X)

$$\sigma{(Y)}$$ es la desviación estándar de (Y)

In [ ]:
df.corr().head()

In [ ]:
# columnas ctes

cte_cols = []

for c in df.select_dtypes(include=np.number):
    
    if df[c].std()==0:
        cte_cols.append(c)

cte_cols

Este resultado se debe al que al eliminar los outliers la desviación estándar de nuestras columnas ha variado con el resultado mostrado arriba, haciendo que ahora estas columnas tengan una varianza de 0 y por lo tanto podemos tratarlas como columnas constantes.
Como hemos dicho antes todas estas columnas no nos aportan valor a nuestro análisis y vamos a eliminarlas

In [ ]:
df = df.drop(columns=cte_cols, axis=1)

In [ ]:
df.corr().head()

Vemos que al volver a calcular la matriz de correlación nos aparece columna con Nulos que no estaba en la lista anterior, siguiendo el mismo procediento vamos a eliminarla

In [ ]:
df = df.drop('phevBlended', axis=1)

In [ ]:
df.corr()

In [ ]:
# criterio de colianelidad

colineales = []

for c in df._get_numeric_data(): # para cada columna numerica ...
    
    for i in range(len(df.corr())): # vamos a recorrer la matriz de correlación ..
        
        if abs(df.corr()[c][i])>0.9 and abs(df.corr()[c][i])<1: 
            # el umbral es arbitrario, abs es el valor absoluto del dato
            #print('columna que estay explorando ', c)
            
            #print('valor de la matriz de correlación ', df.corr()[c][i] )
            colineales.append(c)

colineales = list(set(colineales))

len(colineales)

In [ ]:
colineales

Al igual que pasa con las columnas constantes, la colinealidad nos dice que estas columnas aportan el mismo dato respecto a la que la hemos comparado, por lo que podemos eliminar dicha columna

In [ ]:
df = df.drop(columns=colineales, axis=1)

In [ ]:
df.shape, df_original.shape

### Normalización columna trany

In [ ]:
df.trany.unique()

Vamos utilizar el método apply de pandas para aplicar una función a la columna trany en la que nos cambie el valor por 'MANUAL' si el dato que recibe la función contiene la substring 'Man', 'AUTO' si el dato que recibe contiene la substring 'Au' o que deje el dato tal cual está si no contiene ninguno

In [ ]:
df.trany = df.trany.apply(lambda x: 'MANUAL' if 'Man' in x else ('AUTO' if 'Au' in x else x))

In [ ]:
df.trany.value_counts()

Como solo tenemos un dato 'unknown' vamos a eliminarlo

In [ ]:
df.drop(df[df.trany=='unknown'].index, axis=0, inplace=True)

In [ ]:
df.head()

## Dataframe Final

    Para finalizar vamos a resetear los índices de nuestro df para que todo quede en orden

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.info(memory_usage='deep')

Nuestro dataset ocupa ahora 11,3 MB, pero vamos a ver si podemos cambiar el tipo de dato para ocupe lo menos posible y de esta forma podamos bajar el coste computacional si quisieramos usarlo para entrenar modelos de Machine Learning

In [ ]:
#%pip install tqdm

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
for d in tqdm(['object', 'integer', 'float']):
    for c in tqdm(df.select_dtypes(d).columns):
        if d == 'object':
            df[c]=df[c].astype('category')
        elif d == 'integer':
            df[c] = pd.to_numeric(df[c], downcast='integer')
        elif d == 'float':
            df[c] = pd.to_numeric(df[c], downcast='float')
        else:
            pass

In [ ]:
df.info(memory_usage='deep')

Con esto hemos pasado de 11,3MB a 603,2 KB lo que haría que si usaramos este dataset para entrenar un modelo trabajar de forma mas eficiente

### Para finalizar guardamos nuestro df limpio

In [ ]:
df.to_csv('../../../data/Vehicles_messy_clean.csv', index=False)